In [ ]:
import dask.dataframe as dd
import matplotlib.pyplot as plt
import os
from sklearn.metrics import r2_score
import dask_ml
import dask
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import QuantileTransformer, PowerTransformer

import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*Sending large graph.*")

from dask.distributed import Client, LocalCluster
import dask.multiprocessing

cluster = LocalCluster(processes=True,n_workers=6, threads_per_worker=1)
client = Client(cluster)
import sys
import pickle 

from data_helpers import *
from metrics import *

# crashes bc not enough memory

In [ ]:
folders = [
    'train0_25',
    'train25_50',
    'train50_75',
    'train75_100'
]

# Read Parquet files from each folder into Dask DataFrames
dfs = [dd.read_parquet(folder) for folder in folders]

# Concatenate all DataFrames into a single DataFrame
data = dd.concat(dfs)

In [ ]:
np.random.seed(42)

orig_partitions = [i for i in range(0,int(data.npartitions))]
np.random.shuffle(orig_partitions) #shuffles inplace

trainSep = int(0.7* data.npartitions)
valEnd = data.npartitions #int(0.05* data.npartitions) + trainSep

sampledPartIdxTrain = orig_partitions[0:trainSep]
sampledPartIdxTest  = orig_partitions[trainSep:valEnd]

In [ ]:
train = data.partitions[sampledPartIdxTrain]
test = data.partitions[sampledPartIdxTest]

In [ ]:
import xgboost

In [ ]:
from xgboost import dask as dxgb

In [ ]:
f = 'ptend_q0002_55'

params = {'objective': 'reg:squarederror',
          'max_depth': 4, 'eta': 0.01, 'subsample': 0.5,
          'min_child_weight': 0.5, 'num_boost_round':100}

xgb = dxgb.DaskXGBRegressor(params)
xgb.fit(train[allF], train[f])

In [ ]:


bst = dxgb.train(client, params, train[allF], train[f], num_boost_round=100)

In [ ]:
f = 'ptend_q0002_55'
valSet = lgb.Dataset(test[allF], label=test[f], free_raw_data=False)
trainSet = lgb.Dataset(train[allF], train[f],free_raw_data=False)

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'l2',
    #'num_leaves': 15,
    #'learning_rate': 0.05,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    'verbose': 1
}


gbm = None

gbm = lgb.train(params,
            trainSet,
            num_boost_round=100, 
            valid_sets=valSet,
            #callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=True)],
            init_model=gbm)